In [243]:
from collections import OrderedDict
import pandas as pd
import numpy as np
import plotly
import plotly.express as px
import hijri_converter
from hijri_converter import Hijri, Gregorian

In [244]:
def hijri_to_greg(hijri_date):
    dt_greg = hijri_date.to_gregorian().ctime()
    dt_greg = pd.to_datetime(dt_greg)
    return dt_greg



In [245]:

from hijri_converter import convert
def convert_date(date_string):
    return pd.to_datetime(date_string, errors = 'coerce')

df1 = pd.read_excel("./monthly_target_dates_df_1442.xlsx" , converters={'Reservation Check-out Date':lambda x:Hijri.fromisoformat(x) ,'Reservation Creation Date':lambda x:Hijri.fromisoformat(x),'Reservation Check-in Date':lambda x:Hijri.fromisoformat(x) })
df2 = pd.read_excel("./monthly_target_dates_df_1444.xlsx" ,  converters={'Reservation Check-out Date':lambda x:Hijri.fromisoformat(x) ,'Reservation Creation Date':lambda x:Hijri.fromisoformat(x),'Reservation Check-in Date':lambda x:Hijri.fromisoformat(x)})

In [246]:
df2.drop(df2.index[-1], inplace=True)

In [247]:
df2.insert(1, 'Reservation Creation Date (Greg)', df2['Reservation Creation Date'].apply(lambda x: hijri_to_greg(x)))


In [248]:
def display_full_df(df):
    with pd.option_context('display.max_rows', None, 'display.max_columns', None):
        display(df)

In [249]:
df2.drop(columns=['Reservation Creation Date'], inplace=True)

In [250]:
df2_line = df2.set_index('Reservation Creation Date (Greg)', inplace=False)
#greg_daily_rows_weekly_df.drop(columns=greg_daily_rows_weekly_df.columns[:2], inplace=True)


In [251]:
df2_line

,M1 - Ramadhan,M2 - Shawwal,M3 - Dhu al-Qi’dah,M4 - Dhu al-Hijjah
Reservation Creation Date (Greg),,,,
2022-07-30,0.0,0.0,0.0,0.0
2022-07-31,0.0,0.0,0.0,0.0
2022-08-01,0.0,0.0,0.0,0.0
2022-08-02,0.0,0.0,0.0,0.0
2022-08-03,0.0,0.0,0.0,0.0
...,...,...,...,...
2023-07-14,NaN,NaN,NaN,645.0
2023-07-15,NaN,NaN,NaN,645.0
2023-07-16,NaN,NaN,NaN,NaN


In [252]:
px.line(df2_line)

In [253]:
df2 = df2.melt(id_vars=['Reservation Creation Date (Greg)'], var_name='Target Month', value_name='#Rooms')

In [254]:
df2.fillna(0, inplace=True)
df2

,Reservation Creation Date (Greg),Target Month,#Rooms
0,2022-07-30,M1 - Ramadhan,0.0
1,2022-07-31,M1 - Ramadhan,0.0
2,2022-08-01,M1 - Ramadhan,0.0
3,2022-08-02,M1 - Ramadhan,0.0
4,2022-08-03,M1 - Ramadhan,0.0
...,...,...,...
1411,2023-07-14,M4 - Dhu al-Hijjah,645.0
1412,2023-07-15,M4 - Dhu al-Hijjah,645.0
1413,2023-07-16,M4 - Dhu al-Hijjah,0.0
1414,2023-07-17,M4 - Dhu al-Hijjah,0.0


In [255]:
# display_full_df(df2) 

Bubble chart 

In [256]:
import plotly.express as px

fig = px.scatter(df2, x="Reservation Creation Date (Greg)", y='Target Month',
	         size="#Rooms", color='Target Month',
                 hover_name='Target Month', log_x=False, size_max=80)
fig.show()


Filled Area Plot

In [257]:
import plotly.graph_objects as go


In [258]:
import plotly.express as px

fig = px.area(df2, x="Reservation Creation Date (Greg)", y="#Rooms", color="Target Month",
             pattern_shape="Target Month", pattern_shape_sequence=[".", "x", "+","/"] )

#fig.update_layout(barmode='overlay')

fig.show()

In [259]:
dummy_start_date = "2019-01-01"
dummy_end_date = "2021-10-03"
# date range from start date to end date and random
# column named value using amount of days as shape
dummy_df = pd.DataFrame({
    "ds": pd.date_range(dummy_start_date, dummy_end_date),
    "value": np.random.randint(low=0, high=30,
size=(pd.to_datetime(dummy_end_date) - pd.to_datetime(dummy_start_date)).days + 1,),
})
dummy_df

,ds,value
0,2019-01-01,4
1,2019-01-02,22
2,2019-01-03,21
3,2019-01-04,14
4,2019-01-05,27
...,...,...
1002,2021-09-29,16
1003,2021-09-30,7
1004,2021-10-01,10
1005,2021-10-02,4


In [260]:
from plotly_calplot import calplot

In [261]:
df2

,Reservation Creation Date (Greg),Target Month,#Rooms
0,2022-07-30,M1 - Ramadhan,0.0
1,2022-07-31,M1 - Ramadhan,0.0
2,2022-08-01,M1 - Ramadhan,0.0
3,2022-08-02,M1 - Ramadhan,0.0
4,2022-08-03,M1 - Ramadhan,0.0
...,...,...,...
1411,2023-07-14,M4 - Dhu al-Hijjah,645.0
1412,2023-07-15,M4 - Dhu al-Hijjah,645.0
1413,2023-07-16,M4 - Dhu al-Hijjah,0.0
1414,2023-07-17,M4 - Dhu al-Hijjah,0.0


In [262]:
def get_calplots(df):
    calplots = {}
    for month_name in df['Target Month'].unique():
        df_tmp = df[df['Target Month'] == month_name].drop(columns='Target Month', inplace=False)
        calplots[month_name] = calplot(
            df_tmp, 
            x="Reservation Creation Date (Greg)",
            y="#Rooms",
            colorscale=[
                (0.00, "white"),
                (0.03, "silver"),
                # (0.11, "gray"),
                (0.33, "green"),
                (0.66, "yellow"),
                (1.00, "red")
            ],
            title=month_name,
            gap=2,
            showscale=True,
            years_title=True,
            space_between_plots=0.3,
            month_lines_width=2.5,
        )
        fig.update_layout(
            title_y=0.98, # Adjust the 'y' position (0.0 to 1.0, 0.0 being the bottom, 1.0 being the top)
            xaxis_title="Reservation Creation Date"
        )
    
    return calplots

In [263]:
def get_calplots_separated_by_year(df):
    calplots = {}
    
    for month_name in df['Target Month'].unique():
        
        plots_list = []
        df_tmp = df[
                    (df['Target Month'] == month_name) 
                ].drop(columns='Target Month', inplace=False)
        
        for year in df_tmp['Reservation Creation Date (Greg)'].dt.year.unique():
            df_filtered_tmp = df_tmp[
                    (df_tmp['Reservation Creation Date (Greg)'].dt.year == year)
                ]
            # print(df_filtered_tmp['Reservation Creation Date (Greg)'].iloc[0].month)
            # print(df_filtered_tmp['Reservation Creation Date (Greg)'].iloc[-1].month)
            # start_month = df_filtered_tmp['Reservation Creation Date (Greg)'].iloc[0].month,
            # end_month = df_filtered_tmp['Reservation Creation Date (Greg)'].iloc[-1].month,
            # print(start_month)
            # print(end_month)
            diff_months = df_filtered_tmp['Reservation Creation Date (Greg)'].iloc[-1].month - df_filtered_tmp['Reservation Creation Date (Greg)'].iloc[0].month + 1
            fig = calplot(
                df_filtered_tmp, 
                start_month=df_filtered_tmp['Reservation Creation Date (Greg)'].iloc[0].month,
                end_month=df_filtered_tmp['Reservation Creation Date (Greg)'].iloc[-1].month,
                total_height=200,
                x="Reservation Creation Date (Greg)",
                y="#Rooms",
                colorscale=[
                    (0.00, "white"),
                    (0.03, "silver"),
                    # (0.11, "gray"),
                    (0.33, "green"),
                    (0.66, "yellow"),
                    (1.00, "red")
                ],
                title=month_name,
                gap=2,
                showscale=True,
                years_title=True,
                space_between_plots=0.3,
                month_lines_width=2.5,
            )
            fig.update_layout(
                width=120*diff_months,
                title_y=0.98, # Adjust the 'y' position (0.0 to 1.0, 0.0 being the bottom, 1.0 being the top)
                xaxis_title="Reservation Creation Date"
            )
            plots_list.append(fig)

        calplots[month_name] = plots_list
    
    return calplots

In [264]:
# calplots = get_calplots(df2)
calplots = get_calplots_separated_by_year(df2)

In [265]:
for month_name, figs_list in calplots.items():
    print(month_name)
    if isinstance(figs_list, list):
        for fig in figs_list:
            # fig.update_layout(width=800)
            fig.show()
    else:
        fig = figs_list
        # fig.update_layout(width=800)
        fig.show()

M1 - Ramadhan


M2 - Shawwal


M3 - Dhu al-Qi’dah


M4 - Dhu al-Hijjah
